In [25]:
import numpy as np
import pandas as pd

import datetime

import requests
import json

# Data Crawling

In [30]:
def requestData(stnCode, dateFrom, dateTo):
    url = 'http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList'
    serviceKey = 'QbHyQHFFSGsv2jCWarvHvX07/7vOUd7bU4sUpzk01IIS5UuH1VSVfDBxCZfiYUTqR6XuQiJ0dee/cTEg3cMQqA=='
    
    params = {
            'serviceKey' : serviceKey,
            'pageNo' : '1',
            'numOfRows' : '999',
            'dataType' : 'JSON',
            'dataCd' : 'ASOS',
            'dateCd' : 'DAY',
            'startDt' : dateFrom.strftime('%Y%m%d'),
            'endDt' : dateTo.strftime('%Y%m%d'),
            'stnIds' : stnCode
    }
    return requests.get(url, params=params)

In [31]:
def getData(stnCode, num):
    MAX_REQUEST_SIZE = 999
    
    ret = pd.DataFrame()
    rest = num
    cur = 0
    while rest > 0:
        now = datetime.datetime.now()
        latest = now - datetime.timedelta(days = 1, hours = 10) # 전일 데이터를 당일 10:00부터 확인 가능
        dateFrom = latest - datetime.timedelta(days = rest - 1)
        dateTo   = latest - datetime.timedelta(days = max(rest - MAX_REQUEST_SIZE, 0))
        
        response_json = json.loads(requestData(stnCode, dateFrom, dateTo).text)
        
        try:
            newData = pd.DataFrame(response_json['response']['body']['items']['item'])
            cur += newData.shape[0]
            ret = ret.append(newData, ignore_index=True, sort=False)
            print('관측소 {}의 {}부터 {}까지의 데이터 {}개를 가져왔습니다. ({}/{})'.format(
                stnCode,
                dateFrom.strftime('%Y-%m-%d'),
                dateTo.strftime('%Y-%m-%d'),
                newData.shape[0],
                cur,
                num
            ))
        except KeyError:
            print('관측소 {}의 {}부터 {}까지의 데이터를 가져오는 데 실패했습니다...'.format(
                stnCode,
                dateFrom.strftime('%Y-%m-%d'),
                dateTo.strftime('%Y-%m-%d'),
            ))
        
        rest -= newData.shape[0]
    
    fileName = 'asos_{}_{}_{}.csv'.format(
        stnCode,
        (latest - datetime.timedelta(days = num - 1)).strftime('%Y%m%d'),
        dateTo.strftime('%Y%m%d'))
    path = 'data/' + fileName
    
    ret.to_csv(path)
    print('수집한 데이터를 {} 경로에 파일로 저장했습니다!'.format(path))

# Main

In [32]:
# 지점코드 불러오기 loading station codes
codes = []
with open('지점코드.csv', encoding='utf-8') as file:
    for line in file.readlines():
        codes.append(line.strip().split(','))
        
print('데이터 크롤링을 시작합니다...')
# for code, _, _ in codes:
#     get_data(code, 500)
get_data(90,3652)

데이터 크롤링을 시작합니다...
관측소 90의 2012-06-03부터 2015-02-26까지의 데이터 999개를 가져왔습니다. (999/3652)
관측소 90의 2015-02-27부터 2017-11-21까지의 데이터 999개를 가져왔습니다. (1998/3652)
관측소 90의 2017-11-22부터 2020-08-16까지의 데이터 999개를 가져왔습니다. (2997/3652)
관측소 90의 2020-08-17부터 2022-06-02까지의 데이터 655개를 가져왔습니다. (3652/3652)
수집한 데이터를 data/asos_90_20120603_20220602.csv 경로에 파일로 저장했습니다!
